In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import catboost
import shap
import matplotlib.pyplot as plt 
# 读取player 的数据
player = pd.read_csv("../datafrom200/players.csv")
x_list = player.columns[7:-5]
X = player[x_list]
y = player["value"]

# 划分训练集 train, validation, test : 0.7, 0.15, 0.15
from sklearn.model_selection import train_test_split
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.3, random_state=21)
X_validation, X_test, y_validation, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=23)

In [2]:
gbdt = GradientBoostingRegressor(n_estimators=600).fit(X_train, y_train)
print(gbdt.score(X_test, y_test))

explainer_gbdt = shap.Explainer(gbdt, X_test)
shap_values_gbdt = explainer_gbdt(X)

0.8717510241297542


 97%|=================== | 9906/10186 [00:33<00:00]       

In [3]:
# # Catboost
# cbt = catboost.CatBoostRegressor()
# cbt.fit(X_train, y_train)
# print(cbt.score(X_test, y_test))

# explainer_cbt = shap.Explainer(cbt)
# shap_values_cbt = explainer_cbt(X)

In [ ]:
name_list = ["Harry Kane", "Mohamed Salah", "Kevin De Bruyne", "Bruno Fernandes", "John Stones", "Fernando Llorente Torres", "Kieran Trippier", "Dybala", "Trippier", "Lukaku", "Hakimi", "Robert Lewandowski", "Haaland", "Werner", "Kylian Mbappé Lottin", "Neymar", "Verratti", "Messi Cuccittini"]
for name_str in name_list:
    mask = player["name"].str.contains(name_str)
    row = player[mask].index[0]
    fig = plt.figure()
    # shap.plots.waterfall(shap_values_gbdt[row])
    values = explainer_gbdt.shap_values(X.iloc[row])
    shap.force_plot(explainer_gbdt.expected_value, values, X.iloc[row], matplotlib=True)
    plt.title("the explaining individual predictions of " + name_str)
    fig.tight_layout()
    fig.savefig("../result/shap_value_local/{name}_gbdt.png".format(name=name_str))

    # fig = plt.figure()
    # shap.plots.waterfall(shap_values_cbt[row])
    # plt.title("the explaining individual predictions of " + name_str)
    # fig.tight_layout()
    # fig.savefig("../result/shap_value_local/{name}_cbt.pdf".format(name=name_str))

In [30]:
name_str = "Cristiano Ronaldo dos Santos Aveiro"

mask = player["name"].str.contains(name_str)
row = player[mask].index[0]
values = explainer_gbdt.shap_values(X.iloc[row])
fig = plt.figure()
shap.initjs()
shap.force_plot(explainer_gbdt.expected_value, values, X.iloc[row], matplotlib=True)
fig.tight_layout()


IndexError: index 0 is out of bounds for axis 0 with size 0

In [35]:
name_str = "Cristiano"

mask = player["name"].str.contains(name_str)
player[mask]


,Unnamed: 0,Unnamed: 0.1,name,overall_rating,potential,value,wage,Crossing,Finishing,Heading_Accuracy,...,Penalties,Composure,Defensive_Awareness,Standing_Tackle,Sliding_Tackle,GK_Diving,GK_Handling,GK_Kicking,GK_Positioning,GK_Reflexes
7719,33,33.0,Cristiano Biraghi,80.0,80.0,18500000.0,58000.0,85.0,58.0,70.0,...,73.0,80.0,79.0,77.0,74.0,12.0,6.0,14.0,10.0,7.0
7731,46,46.0,Cristiano Piccini,72.0,72.0,2000000.0,10000.0,67.0,35.0,66.0,...,44.0,76.0,72.0,74.0,72.0,8.0,9.0,5.0,7.0,11.0
8197,36,36.0,Gianluca Cristiano Busio,73.0,84.0,6500000.0,4000.0,54.0,63.0,58.0,...,60.0,73.0,60.0,67.0,58.0,15.0,15.0,6.0,12.0,13.0
